1차 
* IMAGE_SIZE = (520, 300)
*  BATCH_SIZE = 16

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 15.0 MB/s 


In [3]:
!mkdir ./data
!mkdir ./data/weights

In [4]:
!unzip -uq "/content/drive/MyDrive/Project/leaf/235842_작물 병해 분류 AI 경진대회_data.zip" -d'/content/data'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.model_selection import train_test_split

import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.utils import Sequence 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_input
from tensorflow.keras.applications.xception import preprocess_input as xcp_input
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_input
from tensorflow.keras.applications.densenet import preprocess_input as densenet_input
import albumentations as A
import tensorflow_addons as tfa

In [7]:
f1 = tfa.metrics.F1Score(num_classes = 7, average='macro', name = 'f1_score')

In [8]:
train = pd.read_csv('/content/data/train.csv')
test = pd.read_csv('/content/data/test.csv')
sample = pd.read_csv('/content/data/sample_submission.csv')

In [9]:
DIR_PATH = 'content/data'
train['image_path'] = '/' + DIR_PATH + '/' + train['img_path']
test['image_path'] = '/' + DIR_PATH + '/' + test['img_path']
train.drop('img_path', axis=1, inplace=True)
test.drop('img_path', axis=1, inplace=True)

In [10]:
# def show_image(path_list, augmentor = None, ncols = 4, title = None):
#   fig, ax = plt.subplots(figsize=(20, 5), nrows=1, ncols=ncols)

#   for i in range(ncols):
#     image = cv2.cvtColor(cv2.imread(path_list[i]), cv2.COLOR_BGR2RGB)
#     if augmentor is not None:
#       image = augmentor(image=image)['image']
#     ax[i].imshow(image)
#     ax[i].set_title(title)
#     print(image.shape)

# code1 = train[train['disease_code'] == 1]['image_path'].iloc[:4].tolist()
# code2 = train[train['disease_code'] == 2]['image_path'].iloc[:4].tolist()
# code3 = train[train['disease_code'] == 3]['image_path'].iloc[:4].tolist()
# code4 = train[train['disease_code'] == 4]['image_path'].iloc[:4].tolist()

# show_image(code1, augmentor=None, ncols=4, title='CODE1')
# show_image(code2, augmentor=None, ncols=4, title='CODE2')
# show_image(code3, augmentor=None, ncols=4, title='CODE3')
# show_image(code4, augmentor=None, ncols=4, title='CODE4')


In [11]:
# import albumentations as A

# augmentor_01 = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.5),
#     A.ShiftScaleRotate(scale_limit=(0.7, 0.9), p=0.5, rotate_limit=30),
#     A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
#     A.Blur(p=0.2),

# ])

# show_image(code1, augmentor=None, ncols=4, title='CODE1')
# show_image(code2, augmentor=augmentor_01, ncols=4, title='CODE2')

# show_image(code3, augmentor=None, ncols=4, title='CODE3')
# show_image(code4, augmentor=augmentor_01, ncols=4, title='CODE4')

In [12]:
class Dataset(Sequence):
    def __init__(self, image_filenames, labels, image_size=(224, 224), batch_size=64,lamb=0.3, 
                 augmentor=None, shuffle=False, pre_func=None):

        self.image_filenames = image_filenames
        self.labels = labels
        self.image_size = image_size
        self.batch_size = batch_size
        self.lamb = 0.3
        self.augmentor = augmentor
        self.pre_func = pre_func
        self.shuffle = shuffle
        
        if self.shuffle:
          pass

      
    def __len__(self):

        return int(np.ceil(len(self.image_filenames) / self.batch_size))
    


    def __getitem__(self, index):

        image_name_batch = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        
        else: 
            label_batch = None

        image_batch = np.zeros((image_name_batch.shape[0], self.image_size[0], self.image_size[1], 3), dtype='float32')
        
        for image_index in range(image_name_batch.shape[0]):
            
            image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            image = cv2.resize(image, (self.image_size[1], self.image_size[0]))
            if self.pre_func is not None:
                image = self.pre_func(image)
                
            image_batch[image_index] = image
        
      
        return image_batch, label_batch

    def generate_cutmix_image(image_batch, label_batch, beta=1.0):
        W = self.image_size[0]
        H = self.image_size[1]
        cut_rat = np.sqrt(1. - self.lamb)
        cut_w = np.int(W * cut_rat)
        cut_h = np.int(H * cut_rat)
        cx = np.random.randint(W)
        cy = np.random.randint(H)

        bbx1 = np.clip(cx - cut_w // 2, 0, W)
        bby1 = np.clip(cy - cut_h // 2, 0, H)
        bbx2 = np.clip(cx + cut_w // 2, 0, W)
        bby2 = np.clip(cy + cut_h // 2, 0, H)

        lam = np.random.beta(beta, beta)
        rand_index = np.random.permutation(len(image_batch))
        target_a = image_batch_labels
        target_b = image_batch_labels[rand_index]
        image_batch_updated = image_batch.copy()
        image_batch_updated[:, bbx1:bbx2, bby1:bby2, :] = image_batch[rand_index, bbx1:bbx2, bby1:bby2, :]
        
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (image_batch.shape[1] * image_batch.shape[2]))
        label = target_a * lam + target_b * (1. - lam)
      
        return image_batch_updated, label
    
    def on_epoch_end(self):
        if(self.shuffle):

            self.image_filenames, self.labels = sklearn.utils.shuffle(self.image_filenames, self.labels)
        else:
            pass

In [13]:
def get_train_valid(train, valid_size=0.2, random_state=0):
  train_path = train['image_path'].values
  train_label = to_categorical(train['disease_code'])
  tr_path, val_path, tr_label, val_label = train_test_split(train_path, train_label, test_size=valid_size, random_state=random_state, stratify= train_label)
  print('tr_path shape:', tr_path.shape, 'tr_label shape:', tr_label.shape, 'val_path shape:', val_path.shape, 'val_label shape:', val_label.shape)
  return tr_path, val_path, tr_label, val_label

In [14]:
def create_model(model_type='efficientnetb0', in_shape=(224, 224, 3), n_classes=7):
    input_tensor = Input(shape=in_shape)

    if model_type == 'resnet50':
        base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'resnet150':
        base_model = tf.keras.applications.ResNet152V2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'densenet':
        base_model = tf.keras.applications.DenseNet201(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'xception':
        base_model = tf.keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb0':
        base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb1':
        base_model = tf.keras.applications.EfficientNetB1(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb2':
        base_model = tf.keras.applications.EfficientNetB2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb3':
        base_model = tf.keras.applications.EfficientNetB3(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb4':
        base_model = tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb5':
        base_model = tf.keras.applications.EfficientNetB5(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb6':
        base_model = tf.keras.applications.EfficientNetB6(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb7':
        base_model = tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet', input_tensor=input_tensor)
        
    x = base_model.output  

    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)    
    preds = Dense(units=n_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=preds)
    

    return model

In [15]:
def lrfn01(epoch):
  lr_start = 1e-5
  lr_max = 1e-3
  lr_rampup_epochs = 2
  lr_sustain_epochs = 1
  lr_step_decay = 0.7

  def calc_fn(epoch):
    if epoch < lr_rampup_epochs:
      lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
    elif epoch < lr_rampup_epochs + lr_sustain_epochs:
      lr = lr_max
    else:
      lr = lr_max * lr_step_decay ** ((epoch - lr_rampup_epochs - lr_sustain_epochs) // 2)
    
    return lr
  
  return calc_fn(epoch)

def lrfn02(epoch):
  lr_start = 1e-6
  lr_max = 2e-7
  lr_rampup_epochs = 2
  lr_sustain_epochs = 1
  lr_step_decay = 0.75

  def calc_fn(epoch):
    if epoch < lr_rampup_epochs:
      lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
    elif epoch < lr_rampup_epochs + lr_sustain_epochs:
      lr = lr_max
    else:
      lr = lr_max * lr_step_decay ** ((epoch - lr_rampup_epochs - lr_sustain_epochs) // 2)
    
    return lr
  
  return calc_fn(epoch)

lr01_cb = LearningRateScheduler(lrfn01, verbose = 1)
lr02_cb = LearningRateScheduler(lrfn02, verbose = 1)
mcp_cb = ModelCheckpoint(filepath = '/content/data/weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss',mode='min', save_best_only=True, save_weights_only=True, verbose=1 )
ely_cb = EarlyStopping(monitor='val_loss', patience=15)
# tqdm_cb = tfa.callbacks.TQDMProgressBar()

augmentor = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(scale_limit=(0.7, 0.9), p=0.5, rotate_limit=30),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
    A.Blur(p=0.2),
    A.GaussNoise(p=0.3),
    A.Cutout(p=0.6)
])

In [16]:
class Config:
  MODEL_TYPE = 'efficientnetb7'
  IMAGE_SIZE = (520, 400)
  BATCH_SIZE = 4
  N_EPOCHS = 50
  IS_FINE_TUNING = True
  FIRST_EPOCHS = 50
  SECOND_EPOCHS = 100
  FIRST_CB = [lr01_cb,ely_cb]
  SECOND_CB = [lr02_cb,ely_cb] 
  AUGMENTOR = augmentor
  PRE_FUNC = eff_input
  INITIAL_LR = 0.0001
  DEBUG = False

In [17]:
def train_model(train_df, config=Config):
    tr_path, val_path, tr_label, val_label = get_train_valid(train_df, valid_size=0.2, random_state=0)
    
    tr_ds = Dataset(tr_path, tr_label, image_size=config.IMAGE_SIZE, batch_size=config.BATCH_SIZE, 
                          augmentor=config.AUGMENTOR, shuffle=True, pre_func=config.PRE_FUNC)
    val_ds = Dataset(val_path, val_label, image_size=config.IMAGE_SIZE, batch_size=config.BATCH_SIZE, 
                          augmentor=None, shuffle=False, pre_func=config.PRE_FUNC)
    if config.DEBUG:
        tr_image_batch = next(iter(tr_ds))[0]
        val_image_batch = next(iter(val_ds))[0]
        print(tr_image_batch.shape, val_image_batch.shape)
        print(tr_image_batch[0], val_image_batch[0])
        
    print('#######', config.MODEL_TYPE, ' START ########')
    model = create_model(model_type=config.MODEL_TYPE, in_shape=(config.IMAGE_SIZE[0], config.IMAGE_SIZE[1], 3), n_classes=7)
    model.compile(optimizer=Adam(lr=config.INITIAL_LR), loss='categorical_crossentropy', metrics=[f1])
    
    if config.IS_FINE_TUNING:
        print('####### Fine tuning START ########')
        for layer in model.layers[:-4]:
            layer.trainable = False
        
        print('####### Classification Layer START. ########')
        history = model.fit(tr_ds, epochs=config.FIRST_EPOCHS,
                           validation_data=val_ds,
                           callbacks=(config.FIRST_CB), verbose=1)
        
        for layer in model.layers:
            if config.MODEL_TYPE in 'efficientnet':
                if not isinstance(layer, layers.BatchNormalization):
                    layer.trainable = True
            else:
                layer.trainable = True
        
        print('####### Total Layer START ########')
        history = model.fit(tr_ds, epochs=config.SECOND_EPOCHS, 
                           validation_data=val_ds,
                           callbacks=(config.SECOND_CB), verbose=1)
    
    else:
        print('####### Model START ########')
        history = model.fit(tr_ds, epochs=config.N_EPOCHS,
                       validation_data=val_ds,
                       callbacks=(config.FIRST_CB), verbose=1)
        
    return model, history

In [ ]:
model, history = train_model(train, config=Config)

test_path = test['image_path'].values

test_ds = Dataset(image_filenames=test_path, labels=None, image_size=Config.IMAGE_SIZE, batch_size = Config.BATCH_SIZE,
                  augmentor= None, shuffle=False, pre_func=Config.PRE_FUNC)
preds = model.predict(test_ds)

tr_path shape: (200,) tr_label shape: (200, 7) val_path shape: (50,) val_label shape: (50, 7)
####### efficientnetb7  START ########
258088960/258076736 [==============================] - 5s 0us/step
####### Fine tuning START ########
####### Classification Layer START. ########
Epoch 1/50

Epoch 00001: LearningRateScheduler setting learning rate to 1e-05.
50/50 [==============================] - 111s 1s/step - loss: 1.9217 - f1_score: 0.1579 - val_loss: 1.8963 - val_f1_score: 0.1355
Epoch 2/50

Epoch 00002: LearningRateScheduler setting learning rate to 0.000505.
50/50 [==============================] - 64s 1s/step - loss: 1.4354 - f1_score: 0.3780 - val_loss: 0.8239 - val_f1_score: 0.5141
Epoch 3/50

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
50/50 [==============================] - 67s 1s/step - loss: 1.6968 - f1_score: 0.3410 - val_loss: 7.6178 - val_f1_score: 0.1364
Epoch 4/50

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
50/50 [======

In [ ]:
class Config:
  MODEL_TYPE = 'efficientnetb6'
  IMAGE_SIZE = (520, 400)
  BATCH_SIZE = 8
  N_EPOCHS = 50
  IS_FINE_TUNING = True
  FIRST_EPOCHS = 50
  SECOND_EPOCHS = 100
  FIRST_CB = [lr01_cb, ely_cb]
  SECOND_CB = [lr02_cb, ely_cb] 
  AUGMENTOR = augmentor
  PRE_FUNC = eff_input
  INITIAL_LR = 0.0001
  DEBUG = False

model2, history2 = train_model(train, config=Config)
test_ds = Dataset(image_filenames=test_path, labels=None, image_size=Config.IMAGE_SIZE, batch_size = Config.BATCH_SIZE,
                  augmentor= None, shuffle=False, pre_func=Config.PRE_FUNC)

preds2 = model2.predict(test_ds)

In [ ]:
class Config:
  MODEL_TYPE = 'efficientnetb5'
  IMAGE_SIZE = (520, 400)
  BATCH_SIZE = 8
  N_EPOCHS = 50
  IS_FINE_TUNING = True
  FIRST_EPOCHS = 50
  SECOND_EPOCHS = 100
  FIRST_CB = [lr01_cb, ely_cb]
  SECOND_CB = [lr02_cb, ely_cb] 
  AUGMENTOR = augmentor
  PRE_FUNC = eff_input
  INITIAL_LR = 0.0001
  DEBUG = False

model3, history3 = train_model(train, config=Config)
test_ds = Dataset(image_filenames=test_path, labels=None, image_size=Config.IMAGE_SIZE, batch_size = Config.BATCH_SIZE,
                  augmentor= None, shuffle=False, pre_func=Config.PRE_FUNC)
preds3 = model3.predict(test_ds)

In [ ]:
class Config:
  MODEL_TYPE = 'resnet50'
  IMAGE_SIZE = (500, 400)
  BATCH_SIZE = 32
  N_EPOCHS = 50
  IS_FINE_TUNING = True
  FIRST_EPOCHS = 20
  SECOND_EPOCHS = 30
  FIRST_CB = [lr01_cb, ely_cb]
  SECOND_CB = [lr02_cb, ely_cb] 
  AUGMENTOR = augmentor
  PRE_FUNC = densenet_input
  INITIAL_LR = 0.0001
  DEBUG = False
  
model4, history4 = train_model(train, config=Config)
test_ds = Dataset(image_filenames=test_path, labels=None, image_size=Config.IMAGE_SIZE, batch_size = Config.BATCH_SIZE,
                  augmentor= None, shuffle=False, pre_func=Config.PRE_FUNC)
preds4 = model4.predict(test_ds)

In [ ]:
class Config:
  MODEL_TYPE = 'resnet50'
  IMAGE_SIZE = (224, 224)
  BATCH_SIZE = 32
  N_EPOCHS = 50
  IS_FINE_TUNING = True
  FIRST_EPOCHS = 20
  SECOND_EPOCHS = 30
  FIRST_CB = [lr01_cb, ely_cb]
  SECOND_CB = [lr02_cb, ely_cb] 
  AUGMENTOR = augmentor
  PRE_FUNC = resnet_input
  INITIAL_LR = 0.0001
  DEBUG = False
  
model5, history5 = train_model(train, config=Config)
test_ds = Dataset(image_filenames=test_path, labels=None, image_size=Config.IMAGE_SIZE, batch_size = Config.BATCH_SIZE,
                  augmentor= None, shuffle=False, pre_func=Config.PRE_FUNC)
preds5 = model5.predict(test_ds)

In [ ]:
#0.2597
# ratio = 0.4 * preds 
# ratio2 = 0.6 * preds2 
# ratio3 = 0.2 * preds3 
# ratio4 = 0.1 * preds4 

total = (ratio + ratio2 )/2

In [ ]:
code = np.argmax(preds, axis = 1)
sample['disease_code'] = code
sample.to_csv('./submit_submit3.csv', index=False)

In [ ]:
sample

,uid,disease_code
0,20000,5
1,20001,1
2,20002,0
3,20003,0
4,20004,2
...,...,...
4745,24745,0
4746,24746,0
4747,24747,0
4748,24748,1


In [ ]:
def submit_model(test, config=Config):
  test_path = test['image_path'].values
  test_ds = Dataset(image_filenames=test['image_path'].values, labels=None, image_size=config.IMAGE_SIZE, batch_size = config.BATCH_SIZE,
                    augmentor= None, shuffle=False, pre_func=config.PRE_FUNC)
  preds = model.predict(test_ds)
  code = np.argmax(preds, axis = 1)
  sample['disease_code'] = code
  sample.to_csv('./submit_file2.csv', index=False)

  return sample

In [ ]:
sample = submit_model(test)